**Librerías y drivers**


In [22]:
import pandas as pd
import sqlite3
import teradatasql
import openpyxl
import matplotlib.pyplot as plt
from datetime import datetime, date
from datetime import timedelta



**Credenciales y conexión a Teradata**

In [23]:
# Datos de conexión
host = 'tdimpprd.entel.cl'
user = 'P_CFAUNDEZ'
password = 'Carlos_Entel_5052'
database = 'CALIDAD'

 
# Conectar a la base de datos
conn = teradatasql.connect(
    host=host,
    user=user,
    password=password,
    database=database
)
 
cursor = conn.cursor()

In [ ]:

# --- 1. Consultar la tabla `bitacora` para obtener las fechas de entrada y salida de las máquinas ---
query_bitacora = """
SELECT 
    YEAR_ID,
    MONTH_ID,
    POP,
    EVENTO
FROM BITACORA_POP_CFS
WHERE evento IN ('ENTRADA', 'SALIDA')
"""
bitacora_df = pd.read_sql(query_bitacora, conn)

# --- 3. Preparar los datos para la consulta de KPIs ---

def calcular_meses_adyacentes(year, month, evento):
    """Calcula el mes anterior y siguiente para un registro de entrada/salida."""
    dt = date(year, month, 1)
    
    if evento == 'ENTRADA':
        mes_anterior = (dt - pd.DateOffset(months=1)).strftime('%Y%m')
        mes_siguiente = dt.strftime('%Y%m')
    elif evento == 'SALIDA':
        mes_anterior = dt.strftime('%Y%m')
        mes_siguiente = (dt + pd.DateOffset(months=1)).strftime('%Y%m')
    else:
        return None, None #Manejo de error, si el evento no es entrada o salida.
        
    return mes_anterior, mes_siguiente

bitacora_df[['MES_ANTERIOR', 'MES_SIGUIENTE']] = bitacora_df.apply(lambda row: calcular_meses_adyacentes(row['YEAR_ID'], row['MONTH_ID'], row['EVENTO']), axis=1, result_type='expand')
bitacora_df = bitacora_df.dropna(subset=['MES_ANTERIOR', 'MES_SIGUIENTE'])
bitacora_df['MES_ANTERIOR'] = bitacora_df['MES_ANTERIOR'].astype(int)
bitacora_df['MES_SIGUIENTE'] = bitacora_df['MES_SIGUIENTE'].astype(int)

# --- 4. Consulta para obtener los KPIs ---

# Crear una lista para almacenar los resultados de cada máquina
resultados_kpis = []
for index, row in bitacora_df.iterrows():
    anio = row['YEAR_ID']
    mes = row['MONTH_ID']
    pop = row['POP']
    mes_anterior = row['MES_ANTERIOR']
    mes_siguiente = row['MES_SIGUIENTE']

    print(mes_anterior)
    print(mes_siguiente)

    # Convertir 'YYYYMM' a 'YYYY-MM-01'
    def convertir_fecha(mes):
        mes = str(mes)  # Convertir el valor a cadena de texto
        año = mes[:4]  # Los primeros 4 caracteres son el año
        mes_numero = mes[4:]  # Los últimos 2 caracteres son el mes
        return f"{año}-{mes_numero}-01"  # Crear la fecha en formato 'YYYY-MM-DD'

    # Convertir las fechas
    mes_anterior = convertir_fecha(mes_anterior)
    mes_siguiente = convertir_fecha(mes_siguiente)

    # Verificar las fechas convertidas
    print(f"Fecha anterior: {mes_anterior}")
    print(f"Fecha siguiente: {mes_siguiente}")

**Extracción de KPI**

In [ ]:
query_kpis = f"""
SELECT 
    anio,
    mes,
    nombre_poligono,
    -- Calculando el promedio de la "velocidad de bajada" por mes
    AVG((aa.pxq_vdl / aa.qQOS_DownloadThroughput) / 1000) AS promedio_velocidad_bajada,
    SUM(aa.qQOS_DownloadThroughput) as muestras_bajada,
    AVG((aa.pxq_vul / aa.qQOS_UploadThroughput) / 1000) AS promedio_velocidad_subida,
    SUM(aa.qQOS_UploadThroughput) as muestras_subida,
    AVG(aa.pxq_latt / aa.qQOS_LatencyAverage) AS promedio_latencia,
    SUM(aa.qQOS_LatencyAverage) as muestras_latencia,
    aa.Device_SIMServiceProviderBrandName as Operador
FROM (
    SELECT
        YEAR(A.FECHA) AS anio,
        MONTH(A.FECHA) AS mes,
        B.nombre_poligono AS nombre_poligono,
        A.Device_SIMServiceProviderBrandName,
        A.Connection_ServiceProviderBrandName,
        A.Connection_Category,
        A.Connection_Type,
        A.Connection_GenerationCategory,
        A.QOS_DownloadThroughputTestSize,
        A.QOS_UploadThroughputTestSize,
        SUM(CASE WHEN A.QOS_DOWNLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' THEN 1 ELSE 0 END) AS qQOS_DownloadThroughput,
        AVG(CASE WHEN A.QOS_DOWNLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' THEN A.QOS_DOWNLOADTHROUGHPUT END) AS avgQOS_DownloadThroughput,
        SUM(CASE WHEN A.QOS_UPLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' THEN 1 ELSE 0 END) AS qQOS_UploadThroughput,
        AVG(CASE WHEN A.QOS_UPLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' THEN A.QOS_UPLOADTHROUGHPUT END) AS avgQOS_UploadThroughput,
        SUM(CASE WHEN (A.QOS_DOWNLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' OR A.QOS_UPLOADTHROUGHPUTTESTSTATUS = 'COMPLETED') THEN 1 ELSE 0 END) AS qQOS_LatencyAverage,
        AVG(CASE WHEN (A.QOS_DOWNLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' OR A.QOS_UPLOADTHROUGHPUTTESTSTATUS = 'COMPLETED') THEN A.QOS_LATENCYAVERAGE END) AS avgQOS_LatencyAverage,
        SUM(CASE WHEN A.QOS_DOWNLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' THEN 1 ELSE 0 END) * AVG(CASE WHEN A.QOS_DOWNLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' THEN A.QOS_DOWNLOADTHROUGHPUT END) AS pxq_vdl,
        SUM(CASE WHEN A.QOS_UPLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' THEN 1 ELSE 0 END) * AVG(CASE WHEN A.QOS_UPLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' THEN A.QOS_UPLOADTHROUGHPUT END) AS pxq_vul,
        SUM(CASE WHEN (A.QOS_DOWNLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' OR A.QOS_UPLOADTHROUGHPUTTESTSTATUS = 'COMPLETED') THEN 1 ELSE 0 END) * AVG(CASE WHEN (A.QOS_DOWNLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' OR A.QOS_UPLOADTHROUGHPUTTESTSTATUS = 'COMPLETED') THEN A.QOS_LATENCYAVERAGE END) AS pxq_latt
    FROM calidad.TUTELA_ONX_MOBILE_V A
    JOIN calidad.BASE_POLIGONOS_GH7 B 
        ON A.LOCATION_GEOHASH7 = B.GEOHASH
        AND A.FECHA BETWEEN DATE '2024-08-01' AND DATE '2024-10-01'
        AND (A.QOS_DOWNLOADTHROUGHPUTTESTSTATUS = 'COMPLETED' 
        OR A.QOS_UPLOADTHROUGHPUTTESTSTATUS = 'COMPLETED')
        AND A.Device_SIMServiceProviderBrandName IN ('Entel','Movistar','Claro','Wom')
    WHERE A.Connection_Type <> 'WIFI'
        AND A.Connection_Type <> 'CALL_SERVICE_ONLY'
        AND A.Connection_Type <> 'CALL_SERVICE_ONLY_ROAMING'
        AND A.Connection_Type <> 'NO_SERVICE'
        AND (A.QOS_DownloadThroughputTestStatus = 'COMPLETED' 
        OR A.QOS_UploadThroughputTestStatus = 'COMPLETED')
        AND B.tipo IN ('OPERATIVO', 'OPERATIVO_PARCIAL', 'INITIAL_TUNNING', 
                       'OPERATIVO_CON_INTERFERENCIA', 'PREVIO', 'EN_INTERVENCION',
                       'ELIMINADO', 'HALTED', 'OPERATIVO_EVENTO', 
                       'INITIAL_TUNNING_PARCIAL', 'HALTED_FALLA')
    GROUP BY 
        YEAR(A.FECHA),
        MONTH(A.FECHA),
        B.nombre_poligono,
        A.Device_SIMServiceProviderBrandName,
        A.Connection_ServiceProviderBrandName,
        A.Connection_Category,
        A.Connection_Type,
        A.Connection_GenerationCategory,
        A.QOS_DownloadThroughputTestSize,
        A.QOS_UploadThroughputTestSize
) AS aa
GROUP BY 
    anio,
    mes,
    nombre_poligono,
	Operador;
"""
    
df_kpis = pd.read_sql(query_kpis, conn)


conn.close()



C:\Users\P_CFAUNDEZ\AppData\Local\Temp\ipykernel_230192\2714323255.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_kpis = pd.read_sql(query_kpis, conn)


Archivo Excel para PowerBI generado: datos_powerbi.xlsx


**Generar excel para Power Bi**

In [ ]:
# Generar Excel para PowerBI
def generar_excel_powerbi(df):
    # Crear un DataFrame limpio para PowerBI        
    df_powerbi = df.copy()

    # Renombrar columnas para evitar confuciones
    df_powerbi.rename(columns={
        'ANIO': 'ANIO',
        'MES': 'MES',
        'promedio_velocidad_subida': 'Velocidad Descarga (mbps)',
        'muestras_bajada': 'muestras_bajada',
        'promedio_velocidad_bajada': 'Velocidad Subida (mbps)',
        'muestras_subida': 'muestras_subida',
        'promedio_latencia': 'Latencia (ms)',
        'muestras_latencia': 'muestras_latencia',
        'Operador':'Operador'

    }, inplace=True)

    # Guardar el DataFrame en un archivo Excel
    nombre_archivo_excel = f"datos_powerbi.xlsx"
    with pd.ExcelWriter(nombre_archivo_excel, engine='xlsxwriter') as writer:
        df_powerbi.to_excel(writer, index=False, sheet_name='Datos')

        # Formatear la tabla para PowerBI
        workbook = writer.book
        worksheet = writer.sheets['Datos']
        (max_row, max_col) = df_powerbi.shape
        column_settings = [{'header': column} for column in df_powerbi.columns]
        worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings, 'style': 'Table Style Medium 6'})

    print(f"Archivo Excel para PowerBI generado: {nombre_archivo_excel}")

# Generar el Excel para PowerBI
generar_excel_powerbi(df_kpis)
